## Imports

In [ ]:
import os
import h5py
import numpy as np

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dropout, BatchNormalization, Dense, GlobalAveragePooling1D
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## Constants

In [ ]:
DOWNSAMPLING_STEP = 4
NUM_CLASSES = 4

ENCODE_MAP = {
        'rest': 0,
        'motor': 1,
        'memory': 2,
        'math': 3,
    }

## Load data

In [ ]:
def get_dataset_name(filename_with_dir):
    filename_without_dir = filename_with_dir.split('/')[-1]
    temp = filename_without_dir.split('_')[:-1]
    dataset_name = '_'.join(temp)
    return dataset_name

def extract_label(filename, logs=False, encode_mapping=ENCODE_MAP):
    for key in encode_mapping:
        if key in filename:
            if logs: print(f"Mapping label {filename} to {encode_mapping[key]}")
            return encode_mapping[key]
    return encode_mapping['math']


def load_all_data(folder_path, logs=False):
    X = []
    y = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.h5'):
            print(f"Loading {filename}...") if logs else None
            file_path = os.path.join(folder_path, filename)
            with h5py.File(file_path, 'r') as f:
                dataset_name = get_dataset_name(file_path)
                data = f.get(dataset_name)[()]
                label = extract_label(filename)
                print(f"Data shape: {data.shape}, label: {label}") if logs else None
                print("_____"*10)
                X.append(data)       # shape (248, 35624)
                y.append(label)      # 'motor', 'rest', etc.
    X = np.array(X)  # shape (n_samples, 248, 35624)
    y = np.array(y)  # shape (n_samples,)
    return X, y

## Preprocessing

In [ ]:
def z_score_normalize(data):
    # data: shape (248, T)
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True) + 1e-8
    return (data - mean) / std


def downsample(data, step=DOWNSAMPLING_STEP):
    # data: shape (248, T)
    return np.array(data[:, ::step])  # keep every 8th column

In [ ]:
# Specify the folder name and load the data. (Don't push data folders to GitHub!!!)
X_train, y_train = load_all_data("Intra/train", logs=1)

In [ ]:
print(f"X_train shape before preprocessing {np.array(X_train).shape}")  # 3D array: (n_samples, n_channels, n_times)
print(y_train)
X_train = [downsample(z_score_normalize(x)) for x in X_train]
X_train = np.array(X_train)  # shape (n_samples, n_channels, n_times)
print(f"X_train shape after preprocessing {np.array(X_train).shape}")  # 3D array: (n_samples, n_channels, n_times)

## LSTM Model only

In [ ]:
time_steps = X_train.shape[1]
num_channels = X_train.shape[2]


if y_train.ndim == 1:
    y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
print(y_train.shape)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Define LSTM model
lstm_model = Sequential([
    LSTM(128, input_shape=(time_steps, num_channels), return_sequences=True),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(64),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(NUM_CLASSES, activation='softmax')
])

lstm_model.compile(optimizer=Nadam(learning_rate=0.001),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])


In [ ]:
history = lstm_model.fit(
    X_train, y_train,   
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

## Testing and evaluating part

In [ ]:
# Load and preprocess test data
X_test, y_test = load_all_data("Intra/test", logs=0)
print(f"X_test shape before preprocessing: {np.array(X_test).shape}")

X_test = [downsample(z_score_normalize(x), step=4) for x in X_test]
X_test = np.array(X_test)
print(f"X_test shape after preprocessing: {np.array(X_test).shape}")

# One-hot encode labels
y_test = to_categorical(y_test, num_classes=4)

In [ ]:
# Evaluate
test_loss, test_acc = lstm_model.evaluate(X_test, y_test, verbose=1)
print("Test accuracy:", test_acc)

In [ ]:
# Plot accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Predict class probabilities
y_pred_probs = lstm_model.predict(X_test)
# Convert one-hot back to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["rest", "motor", "memory", "math"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()
